Input:
- Model Parameter: population, alpha,
- Ggf. Calculation r-value (traditional, neural network, lstm)

Output:
- Calculated/Predicted Observation

In [1]:
import torch
from torch import optim
from torch import distributions
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import imageio
import os

In [110]:
#%cd master-thesis

In [2]:
%cd ..

/home/anuar/PycharmProjects/master-thesis


In [48]:
def epidim_model(population, R, initial_newly_infected, alpha):
        n = R.size()[0]
        dtype = torch.float32
        #device = torch.device("cuda")
        df_si_dd = pd.read_csv('data/si_and_dd_covid19model.csv', sep=',', decimal='.', encoding='utf-8', dtype={'serial_interval': np.float64, 'delay_distr': np.float64})
        si = torch.tensor(df_si_dd['serial_interval'][0:n],dtype=dtype)
        dd =  torch.tensor(df_si_dd['delay_distr'][0:n],dtype=dtype)
        # Seed initial Infections
        newly_infected = torch.zeros(n, dtype=dtype)
        # number of newly infected
        cumulative_infected = torch.zeros(n,dtype=dtype)
        #cumulative number of infected
        immune = torch.zeros(n,dtype=dtype)
        St = torch.ones(n,dtype=dtype)  # fraction of susceptible population
        effectively_infectious = torch.zeros(n,dtype=dtype)
        cero = torch.tensor(0.,dtype=dtype)

        num_init = initial_newly_infected.size
        newly_infected[0:num_init] = torch.tensor(initial_newly_infected, dtype=dtype)
        cumulative_infected[1:num_init] = torch.cumsum(newly_infected[0:num_init - 1].clone(), dim=0)

        immune[0:num_init] = cumulative_infected[0:num_init].clone() #+ vaccine_inmune[0:num_impute].clone()
        # TODO: Add vaccine

        for t in range(num_init, n):
                # Update cumulative newly_infected
                cumulative_infected[t] = cumulative_infected[t - 1] + newly_infected[t - 1]
                immune[t] = cumulative_infected[t] #+ vaccine_inmune[t]
                # Adjusts for portion of pop that are susceptible
                St[t] = torch.maximum(population - immune[t], cero) / population
                # effective number of infectous individuals
                ni_temp = newly_infected[:t].view(1, 1, -1).clone()
                si_temp = torch.flip(si, (0,))[-t:].view(1, 1, -1)
                effectively_infectious[t] = torch.nn.functional.conv1d(ni_temp, si_temp)

                newly_infected[t] = St[t].clone() * R[t] * effectively_infectious[t].clone()

        # calc hospitalizations
        # Initialize expected_daily_hospit
        expected_daily_hospit = torch.zeros(n)  # expected number of daily hospitalizations

        # calculate expected number of hospitalizations
        expected_daily_hospit[0] = (1e-15) * newly_infected[0]
        for t in range(1, n):
                ni_temp = newly_infected[:t].view(1, 1, -1)
                dd_temp = torch.flip(dd, (0,))[-t - 1:-1].view(1, 1, -1)
                expected_daily_hospit[t] = alpha * torch.nn.functional.conv1d(ni_temp, dd_temp)

        return {'St': St, 'newly_infected': newly_infected, 'cumulative': cumulative_infected, 'hospitalization': expected_daily_hospit}

# Tests

In [50]:
import plotly.express as px
import pandas as pd

population = 100000
n = 300
R = 3*torch.rand(n)
init = np.array([3,3,3,3,3])
alpha = 0.8
observations = epidim_model(population,R, init, alpha)

observations_df = pd.DataFrame(observations)
px.line(observations_df, y=observations_df.columns)